In [3]:
import torch
import torchbearer
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import numpy as np
import pickle
from torchbearer.callbacks import LiveLossPlot
from itertools import product
from scipy.ndimage import gaussian_filter
import numpy as np
from matplotlib import pyplot as plt
import random
import torch.nn.functional as F
from torch import nn

In [4]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

In [13]:
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)
#split the data
trainset.data = trainset.data[0:27105]
trainset.targets = trainset.targets[0:27105]
# # print(trainset.targets[0:11905])
#for distillation
trainloaderforDistillation = DataLoader(trainset, batch_size=1000, shuffle=True)
testloaderforDistillation = DataLoader(testset, batch_size=1000, shuffle=True)

In [14]:
class BetterCNNforDistillation(nn.Module):
    def __init__(self):
        super(BetterCNNforDistillation, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (5, 5), padding=0)
        self.conv2 = nn.Conv2d(32, 64, (3, 3), padding=0)
        self.fc1 = nn.Linear(64 * 5**2, 1024)
        self.fc2 = nn.Linear(1024, 50)
        self.fc3 = nn.Linear(50, 10)

            
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = self.conv2(out)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = F.dropout(out, 0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

In [15]:
#train distillion network

modelDistillation = BetterCNNforDistillation().to(device)
loss_function = nn.CrossEntropyLoss()
# optimiser = optim.Adam(model.parameters())
optimiser = optim.SGD(modelDistillation.parameters(), lr=0.1, momentum=0.5)
running_loss = 0.0

#teacher network
for epoch in range(2):
    running_loss = 0.0
    for data in trainloaderforDistillation:
        # get the inputs
        inputs, labels = data
        inputs = torch.reshape(inputs, [-1, 1,28,28])
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimiser.zero_grad()

        # forward + loss + backward + optimise (update weights)
        outputs = modelDistillation(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
        
        #update the label for modelDistillation
        if epoch == 1:       
            predictions = torch.argmax(outputs, 1)
            labels = predictions
            
        
        # keep track of the loss this epoch
        running_loss += loss.item()
    print("Epoch %d, loss %4.2f" % (epoch, running_loss))
print('**** Finished Training ****')


Epoch 0, loss 63.46
Epoch 1, loss 57.52
**** Finished Training ****


In [16]:
#student network
for epoch in range(2):
    running_loss = 0.0
    for data in trainloaderforDistillation:
        # get the inputs
        inputs, labels = data
        inputs = torch.reshape(inputs, [-1, 1,28,28])
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimiser.zero_grad()

        # forward + loss + backward + optimise (update weights)
        outputs = modelDistillation(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step() 
        
        # keep track of the loss this epoch
        running_loss += loss.item()
    print("Epoch %d, loss %4.2f" % (epoch, running_loss))
print('**** Finished Training ****')

Epoch 0, loss 29.56
Epoch 1, loss 8.95
**** Finished Training ****


In [18]:
modelDistillation.eval()

# Compute the model accuracy on the test set
correct = 0
total = 0

for data in testloaderforDistillation:
    inputs, labels = data
    inputs = torch.reshape(inputs, [-1, 1,28,28])
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = modelDistillation(inputs)
    predictions = torch.argmax(outputs, 1)
    correct += (predictions == labels).sum().float()
    total += len(labels)
print('Test Accuracy: %2.2f %%' % ((100.0 * correct) / total))
torch.save(modelDistillation,'save_dis.pkl')

Test Accuracy: 89.93 %


d:\Anaconda3\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type BetterCNNforDistillation. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
